In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.pandas as pd
import pyspark

In [21]:
spark = SparkSession.builder \
        .appName("Relational Sample PySpark SQL Server Connection") \
        .config("spark.jars", "../jdbc/mssql-jdbc-12.6.1.jre8.jar") \
        .getOrCreate()

In [22]:
def spark_read_from_db(table_name):
    server_name = "mssql"
    port = "1433"
    database_name = "Data"
    url = f"jdbc:sqlserver://{server_name}:{port};databaseName={database_name}"
    username = "SA"
    password = "YourStrongPassword123"
    df = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .option("encrypt", "false") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .load()
    return df

In [16]:
properties = spark_read_from_db("ARProperties")
clients = spark_read_from_db("ARClients")
properties_clients = spark_read_from_db("PropertiesClients")

In [17]:
join_condition_properties = properties["id"] == properties_clients["id_prop"]
join_condition_clients = clients["id"] == properties_clients["id_client"]

joined_df = properties_clients.join(clients, join_condition_clients) \
                              .join(properties, join_condition_properties)
columns_to_drop = ["id"]
joined_df = joined_df.drop(*columns_to_drop)

In [19]:
joined_df.show(10)

+-------+---------+----------------+---------+-------------+--------------------+--------------------+-----------------+--------------------+------------------+------+---------+----------+----------+----------+--------------+--------------+---------+--------------------+--------------------+--------+------+-----+--------+---------+-------------+---------------+--------+--------+------------+--------------------+-------------+--------------+
|id_prop|id_client|      first_name|last_name|date_of_birth|               email|             address|annual_income_usd|                 job|number_of_children|   sex|education|start_date|  end_date|created_on|      latitude|     longitude|  country|            province|                city|district|estate|rooms|bedrooms|bathrooms|surface_total|surface_covered|   price|currency|price_period|               title|property_type|operation_type|
+-------+---------+----------------+---------+-------------+--------------------+--------------------+--------